# 02 - Preprocesamiento y Feature Engineering

**Objetivo:** En este notebook, preparamos el dataset para el entrenamiento de modelos de Machine Learning. Las tareas incluyen: selección de características basada en el EDA, separación de datos en conjuntos de entrenamiento y prueba, y escalado de las características numéricas.

## 1. Carga de Datos e Importaciones

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [6]:
# Cargar el dataset optimizado
df = pd.read_parquet(r'../data/processed/cic_ids_2017_optimized.parquet')

## 2. Selección de Características (Feature Selection)

Basado en el Análisis Exploratorio de Datos (EDA), tomaremos decisiones sobre qué características conservar.

In [7]:
# Selección Manual (Basada en Correlación)
# En el EDA, vimos una correlación muy alta (0.89) entre 'Flow Duration' y 'Flow IAT Mean'.
# Decidimos eliminar 'Flow IAT Mean' para reducir la redundancia.
df.drop(columns = ['Flow IAT Mean'], inplace = True)

## 3. Separación de Datos

Separamos nuestras variables predictoras (**X**) de nuestra variable objetivo (**y**).

In [8]:
X = df.drop('Label', axis = 1)
y = df['Label']

## 4. División en Conjuntos de Entrenamiento y Prueba (Train/Test Split)

Dividimos el dataset. Es **crucial** hacer esto **antes** de cualquier otro preprocesamiento (como el escalado) para evitar la fuga de datos (data leakage). Usamos **stratify = y** para asegurar que la proporción de clases sea la misma en ambos conjuntos, lo cual es vital para datasets desbalanceados.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)